<!-- Projeto Desenvolvido na Data Science Academy - www.datascienceacademy.com.br -->

# <font color='blue'>Data Science Academy</font>

# <font color='blue'>Processamento de Linguagem Natural com Transformers</font>

## <font color='blue'>Projeto 7</font>

## <font color='blue'>Aplicando LLM Para Análise de Texto em Documentos Legais</font>

## Instalando e Carregando Pacotes

In [1]:
# Versão da Linguagem Python
from platform import python_version
print('Versão da Linguagem Python Usada Neste Jupyter Notebook:', python_version())

Versão da Linguagem Python Usada Neste Jupyter Notebook: 3.10.12


In [2]:
# Para atualizar um pacote, execute o comando abaixo no terminal ou prompt de comando:
# pip install -U nome_pacote

# Para instalar a versão exata de um pacote, execute o comando abaixo no terminal ou prompt de comando:
# !pip install nome_pacote==versão_desejada

# Depois de instalar ou atualizar o pacote, reinicie o jupyter notebook.

# Instala o pacote watermark.
# Esse pacote é usado para gravar as versões de outros pacotes usados neste jupyter notebook.
!pip install -q -U watermark

In [3]:
# Instalando os pacotes
!pip install -q -r requirements.txt

In [4]:
# Imports
import langchain
import textwrap
from langchain import PromptTemplate
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
import warnings
warnings.filterwarnings('ignore')

In [5]:
%env TOKENIZERS_PARALLELISM=true

env: TOKENIZERS_PARALLELISM=true


A variável TOKENIZERS_PARALLELISM é frequentemente usada em conjunção com bibliotecas de PLN. Ela controla se os tokenizadores (que convertem texto em tokens ou unidades menores para processamento por modelos de linguagem) utilizam paralelismo ou não.

Quando definido como true, permite que os tokenizadores operem em paralelo, o que pode acelerar certas operações, mas também pode levar a problemas de conflito em ambientes onde o paralelismo não é bem gerenciado.

Se definido como false, desativa o paralelismo, o que pode ser útil para evitar problemas de concorrência em ambientes com múltiplas threads ou processos.

In [6]:
# Versões dos pacotes usados neste jupyter notebook
%reload_ext watermark
%watermark -a "Data Science Academy" --iversions

Author: Data Science Academy

langchain: 0.1.4



In [7]:
# Função para carregar o pdf
def dsa_carrega_pdf(file_path):

    # Cria uma instância da classe PyMuPDFLoader, passando o caminho do arquivo PDF como argumento.
    loader = PyMuPDFLoader(file_path=file_path)

    # Usa o método 'load' do objeto 'loader' para carregar o conteúdo do PDF.
    # Isso retorna um objeto ou uma estrutura de dados contendo as páginas do PDF com seu conteúdo.
    docs = loader.load()

    # Retorna o conteúdo carregado do PDF.
    return docs

In [8]:
# Função para dividir os documentos em vários pedaços (chunks)
def dsa_split_docs(documents, chunk_size = 1000, chunk_overlap = 20):

    # Cria uma instância da classe RecursiveCharacterTextSplitter.
    # Esta classe divide textos longos em pedaços menores (chunks).
    # 'chunk_size' define o tamanho de cada pedaço, e 'chunk_overlap' define a sobreposição entre pedaços consecutivos.
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = chunk_size, chunk_overlap = chunk_overlap)

    # Utiliza o método 'split_documents' do objeto 'text_splitter' para dividir o documento fornecido.
    # 'documents' é uma variável que contém o texto ou conjunto de textos a serem divididos.
    chunks = text_splitter.split_documents(documents = documents)

    # Retorna os pedaços de texto resultantes da divisão.
    return chunks

In [9]:
# Carrega o modelo de embeddings
def dsa_carrega_embedding_model(model_path, normalize_embedding=True):

    # Retorna uma instância da classe HuggingFaceEmbeddings.
    # 'model_name' é o identificador do modelo de embeddings a ser carregado.
    # 'model_kwargs' é um dicionário de argumentos adicionais para a configuração do modelo, neste caso, definindo o dispositivo para 'cpu'.
    # 'encode_kwargs' é um dicionário de argumentos para o método de codificação, aqui especificando se os embeddings devem ser normalizados.
    return HuggingFaceEmbeddings(model_name = model_path,
                                 model_kwargs = {'device':'cpu'},
                                 encode_kwargs = {'normalize_embeddings': normalize_embedding})


In [10]:
# Função para criar embeddings usando FAISS
def dsa_cria_embeddings(chunks, embedding_model, storing_path = "modelo/vectorstore"):

    # Cria um 'vectorstore' (um índice FAISS) a partir dos documentos fornecidos.
    # 'chunks' é a lista de segmentos de texto e 'embedding_model' é o modelo de embedding utilizado para converter texto em embeddings.
    vectorstore = FAISS.from_documents(chunks, embedding_model)

    # Salva o 'vectorstore' criado em um caminho local especificado por 'storing_path'.
    # Isso permite a persistência do índice FAISS para uso futuro.
    vectorstore.save_local(storing_path)

    # Retorna o 'vectorstore' criado, que contém os embeddings e pode ser usado para operações de busca e comparação de similaridade.
    return vectorstore

In [11]:
# Criando a chain
def dsa_load_qa_chain(retriever, llm, prompt):

    # Retorna uma instância da classe RetrievalQA.
    # Esta função lida com a cadeia de processos envolvidos em um sistema de Question Answering (QA).
    # 'llm' refere-se ao modelo de linguagem de grande escala (como um modelo GPT ou BERT).
    # 'retriever' é um componente usado para recuperar informações relevantes (como um mecanismo de busca ou um retriever de documentos).
    # 'chain_type' define o tipo de cadeia ou estratégia usada no processo de QA. Aqui, está definido como "stuff", um placeholder para um tipo real.
    # 'return_source_documents': um booleano que, quando True, indica que os documentos fonte (ou seja, os documentos de onde as respostas são extraídas) devem ser retornados juntamente com as respostas.
    # 'chain_type_kwargs' é um dicionário de argumentos adicionais específicos para o tipo de cadeia escolhido. Aqui, está passando 'prompt' como argumento.
    return RetrievalQA.from_chain_type(llm = llm,
                                       retriever = retriever,
                                       chain_type = "stuff",
                                       return_source_documents = True,
                                       chain_type_kwargs = {'prompt': prompt})


In [12]:
# Função para obter as respostas do LLM (Large Language Model)
def dsa_get_response(query, chain):

    # Invoca a 'chain' (cadeia de processamento, um pipeline de Question Answering) com a 'query' fornecida.
    # 'chain' é uma função que recebe uma consulta e retorna uma resposta, utilizando o LLM.
    response = chain({'query': query})

    # Utiliza a biblioteca textwrap para formatar a resposta. 'textwrap.fill' quebra o texto da resposta em linhas de largura especificada (100 caracteres neste caso),
    # tornando mais fácil a leitura em ambientes como o Jupyter Notebook.
    wrapped_text = textwrap.fill(response['result'], width=100)

    # Imprime o texto formatado
    print(wrapped_text)

Crie sua API na OpenAI e coloque na célula abaixo.

https://platform.openai.com/

https://platform.openai.com/api-keys

https://platform.openai.com/docs/quickstart?context=python

In [13]:
# Definindo a API da OpenAI
llm_api = OpenAI(openai_api_key="coloque-aqui-sua-api")

https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2

In [14]:
# Carrega o modelo Embedding
embed = dsa_carrega_embedding_model(model_path = "all-MiniLM-L6-v2")

In [15]:
# Carrega o arquivo pdf
docs = dsa_carrega_pdf(file_path = "CF88_Livro_EC91_2016.pdf")

In [16]:
# Divide o arquivo em chunks
documents = dsa_split_docs(documents = docs)

In [17]:
# Cria o vectorstore
vectorstore = dsa_cria_embeddings(documents, embed)

In [18]:
# Converte o vectorstore para um retriever
retriever = vectorstore.as_retriever()

In [19]:
template = """
### System:
Você é um assistente jurídico. Você tem que responder as perguntas do usuário \
usando apenas o contexto fornecido a você. Se você não sabe a resposta, \
apenas diga que você não sabe. Não tente inventar uma resposta.

### Context:
{context}

### User:
{question}

### Response:
"""

In [20]:
# Criando o prompt a partir do template
prompt = PromptTemplate.from_template(template)

In [21]:
# Criando a chain (pipeline)
dsa_chain = dsa_load_qa_chain(retriever, llm_api, prompt)

In [22]:
dsa_get_response("Quais os requisitos para aposentadoria pelas normas estabelecidas para o \
                 regime geral de previdência social?", dsa_chain)

Os requisitos para a aposentadoria pelas normas estabelecidas para o regime geral de previdência
social são definidos pela legislação vigente, sendo necessário ter contribuído para o sistema por um
determinado período de tempo e atingir a idade mínima estabelecida. Além disso, existem requisitos
específicos para aposentadorias especiais e por invalidez.


In [23]:
dsa_get_response("O ensino e direito à educação têm como base quais princípios?", dsa_chain)

Os Estados Partes asseguram o direito à educação para pessoas com deficiência com base na igualdade
de oportunidades, incluindo o pleno desenvolvimento do potencial humano e do senso de dignidade e
autoestima, além do fortalecimento do respeito pelos direitos humanos e pela diversidade humana.


In [24]:
dsa_get_response("O que acontece se uma medida provisória não for apreciada em até quarenta \
                    e cinco dias contados de sua publicação?", dsa_chain)

Se a medida provisória não for apreciada em até quarenta e cinco dias contados de sua publicação,
ela entrará em regime de urgência, ficando sobrestadas todas as demais deliberações legislativas da
Casa em que estiver tramitando, até que se ultime a votação.


# Fim